<a href="https://colab.research.google.com/github/rennydoig/ssc22-case-comp/blob/main/Daisy_dir/Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import csv
!pip install geopandas rtree
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/2022 SSC Case Study')
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/2022 SSC Case Study


In [ ]:
# Read data
df = pd.read_csv("ookla-canada-speed-tiles.csv")

# Check data shape
print("data shape:{shape}".format(shape=df.shape),"\n", "data columns:{columns}".format(columns=list(df.columns))) 

data shape:(2751464, 21) 
 data columns:['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'year', 'quarter', 'conn_type', 'PRUID', 'PRNAME', 'CDUID', 'CDNAME', 'DAUID', 'SACTYPE', 'DA_POP', 'PCUID', 'PCNAME', 'PCTYPE', 'PCCLASS', 'geometry']


In [ ]:
def makeData(df, aggregate_by, conType, value):
    """ Make an appropriate data frame for prediction """

    '''
    df: orginal data frame from SSC
    aggregate_by: 'quadkey' or 'DAUID' or 'CDNAME' or "PRNAME"
    conType: boolen, has the option to seperate connect type (e.g. fixed or mobile)
    value: 'avg_d_mbps' or 'avg_u_mbps' or 'avg_lat_ms'
    '''

    # Change kbps to mbps
    df['avg_d_mbps'] = df['avg_d_kbps'] / 1000
    df['avg_u_mbps'] = df['avg_u_kbps'] / 1000

    # Combine "year" and "quarter" and create a new column called "Date"
    df['Date'] = ["-".join(i) for i in zip(df['year'].astype(str), df['quarter'])]

    # Filter only rural areas: rows with NaN for either PCUID, PCNAME, PCTYPE, or PCCLASS
    cond = (df['PCUID'].isna()) | (df['PCNAME'].isna()) | (df['PCTYPE'].isna()) | (df['PCCLASS'].isna())
    df_rural = df[cond]
  
    # Aggregate data
    dat = dataAggregation(df_rural,aggregate_by,conType)

    # Transform the data: rows -> aggregate_by and columns -> Date
    if conType:
      output_data = dat.pivot_table(values=value,index=[aggregate_by],columns=['conn_type','Date'],aggfunc='mean')
    else:
      output_data = dat.pivot_table(values=value,index=[aggregate_by],columns=['Date'],aggfunc='mean')
    
    # Append 'tests' and 'DA_POP'
    output_data[['tests','DA_POP']] = dat.groupby(aggregate_by).agg({'tests':'sum','DA_POP':'sum'})

    return output_data


def dataAggregation(df, aggregate_by, conType):
    """ Aggregate data by tile, dissemination area, census division or province """

    # Weighted average of 'avg_d_mbps', 'avg_u_mbps' and 'avg_lat_ms', weight = 'tests'
    wm = lambda x: np.average(x, weights=df.loc[x.index, 'tests'])

    if conType:
      dat = df.groupby([aggregate_by,'Date','conn_type'],as_index=False).agg({"avg_u_mbps":wm,
                                                                              "avg_d_mbps":wm,
                                                                              "avg_lat_ms":wm,
                                                                              "tests":"sum",
                                                                              "DA_POP":"sum"})
    else: 
      dat = df.groupby([aggregate_by,'Date'],as_index=False).agg({"avg_u_mbps":wm,
                                                                  "avg_d_mbps":wm,
                                                                  "avg_lat_ms":wm,
                                                                  "tests":"sum",
                                                                  "DA_POP":"sum"})
    return dat

In [ ]:
# Final output data will look like
dat = makeData(df, 'CDNAME', True, 'avg_d_mbps')
dat

conn_type              fixed                                                 \
Date                 2019-Q1    2019-Q2     2019-Q3     2019-Q4     2020-Q1   
CDNAME                                                                        
Abitibi            11.462567  13.554874   12.990781   16.801290   14.693715   
Abitibi-Ouest      15.769587  15.541620   19.051350   16.881538   16.716508   
Acton              40.074545  29.238450   32.274028   90.410018   87.360752   
Alberni-Clayoquot  64.842903  54.325963   38.189325   67.063167  109.383307   
Albert             41.057450  16.897223   38.484176   72.213592   65.870083   
...                      ...        ...         ...         ...         ...   
Wellington         28.430102  27.210868   30.917539   33.692628   31.876176   
Westmorland        94.746735  56.862206  101.517383  110.358326  121.959130   
Yarmouth           58.014153  54.932696   54.492475   65.283673   69.589351   
York               86.053464  78.115540   92.005492  109.065318  117.554432   
Yukon              39.515250  18.690258   23.918764   34.644554   32.557076   

conn_type                                                                      \
Date                  2020-Q2     2020-Q3     2020-Q4     2021-Q1     2021-Q2   
CDNAME                                                                          
Abitibi             15.610093   39.297977   72.982751   59.796979   68.852650   
Abitibi-Ouest       21.906001   23.159099   42.930830   30.968243   39.336971   
Acton               65.469779   51.885451   77.248615   85.951806   83.869586   
Alberni-Clayoquot   68.881146   64.925728   94.042159   99.393962   94.913226   
Albert              62.957604   75.554547   93.548210   75.259242   91.438707   
...                       ...         ...         ...         ...         ...   
Wellington          25.728160   33.731816   51.590634   51.020585   55.596436   
Westmorland        103.641952  114.795490  155.669593  166.953091  163.200988   
Yarmouth            62.809892   60.101993   70.966971   71.912654   91.730810   
York                97.125635  105.591318  110.488218  114.012489  133.765207   
Yukon               34.397581   33.937172   47.552216   34.292679   42.344253   

conn_type          ...     mobile                                              \
Date               ...    2020-Q1    2020-Q2    2020-Q3    2020-Q4    2021-Q1   
CDNAME             ...                                                          
Abitibi            ...  20.869593  30.754828  24.817265  24.934930  35.478649   
Abitibi-Ouest      ...  17.318108  17.847725  26.620284  30.475953  30.148667   
Acton              ...  80.165250  53.667632  45.544689  41.927529  40.603889   
Alberni-Clayoquot  ...  34.382714  79.694852  59.133625  34.195846  66.818438   
Albert             ...  15.525385  40.700706  20.676938  32.352469  21.711333   
...                ...        ...        ...        ...        ...        ...   
Wellington         ...  51.261216  35.733502  37.077029  59.936080  49.347517   
Westmorland        ...  58.646382  75.514029  75.446341  88.832896  57.638135   
Yarmouth           ...  33.289500  23.985200  26.511091  33.039867  21.224429   
York               ...  63.452748  53.453679  72.271385  94.965056  68.952483   
Yukon              ...  33.984923  11.760500  16.271588  53.668571  18.731000   

conn_type                                             tests      DA_POP  
Date                 2021-Q2    2021-Q3     2021-Q4                      
CDNAME                                                                   
Abitibi            13.564721  17.738018   54.506764   18784   2239430.0  
Abitibi-Ouest      63.525188  22.652034   45.520000   16094   1472200.0  
Acton              58.961500  49.287700  103.575286    6335    769420.0  
Alberni-Clayoquot  82.080143  48.941393  118.612333    7274    979335.0  
Albert             23.249443  42.621103   45.330980    6901    769885.0  
...                      ...   